In [1]:
# 임포트
# 임포트
import pandas as pd
import numpy as np
import langchain 
import requests
from bs4 import BeautifulSoup

# 서제스트 키워드 긁어오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# 임포트 2
from create_URL import createDaumURL
from create_URL import createGoogleURL
from create_URL import createYoutubeURL
from create_URL import createNaverURL

from getOpenKeywords import getOpenKeywords
from keywordFormatting import keywordFormatting
from seleniumClickAndCrawling import seleniumClickAndCrawling
from countDaumADs import countDaumADs
from count_GoogleSearchVolume import countGoogleSearchVolume
from checkGoogleSectionOrder import checkGoogleSectionOrder
from getGoogleOpenKeywords import getGoogleOpenKeywords
from limitLength import limitLength

In [3]:
# config _ format 설정
# config
# 수집데이터
keywords = {}

providers = ['daumOpenKeywords',
'daumSuggestKeywords',
'googleSuggestKeywords',
'googleOpenKeywords',
# 'youtubeSuggestKeywords',
]
benchmarks = ['searchVolume','countDaumADs','tistoryRank','provider']

searchKeywordFormat = {}

# searchKeywordFormat = {
        # 'daumOpenKeywords' : {},
        # 'daumSuggestKeywords' : {},
        # 'naverOpenKeywords' : {},
        # 'googleSuggestKeywords' : {},
        # 'googleOpenKeywords':{},
        # 'youtubeSuggestKeywords':{},
        # 'mergedKeywords' : {},
        # }    

keywordValuationFormat = {
    'searchVolume' : '',
    'countDaumADs' : '',
    'tistoryRank' : '',    
    'provider' : '',
    'depth' : ''  
}


In [4]:
# config _ CSSSelector설정

## 오픈키워드 선택자
# daumOpenKeywordsTag = ['#netizen_lists_top span', '#recomm_lists_top a'] # 관련키워드
daumOpenKeywordsTag =  'a.keyword'# 관련키워드

naverOpenKeywordsTag = 'a.title_link'

# 서제스트키워드 선택자
daumClickElementCSSQuery = '#q.tf_keyword'
googleClickElementCSSQuery = '#searchform textarea.gLFyf'
youtubeClickElementCSSQuery = 'input#search'

daumDataCSSQuery = "span.txt_query"
googleDataCSSQuery = 'div.wM6W7d span'
youtubeDataCSSQuery = '#sbse0 > div.sbqs_c > b'


getGoogleOpenKeywordCSSQuery = 'div.s75CSd.u60jwe.r2fjmd.AB4Wff'
# 다음광고 선택자
DAUM_AD_normal = '.list_ad' #와이드광고, 프리미엄광고
DAUM_AD_special = '#splinkColl .list_info.mg_cont.clear' # 스페셜광고
DAUM_REL_KEYWORDS = '#netizen_lists_top span' # 관련키워드

# 검색량 선택자
# config(countGoogleSearchVolume)
volumeCSSQuery = 'div#result-stats'

# 순서 선택자
# config(checkGoogleSectionOrder)
rankCSSQuery = 'div#rso.dURPMd > div.MjjYud a'

In [5]:
# # 해인이 config

# # 시스템 설정
# debug = False # 디버그모드 설정 :
# maxDepth = 2 # depth 설정
# limit = 4 # 크롤링 키워드 갯수한계 설정

# # 내용 설정
# targets = [] # 글감 작성
# streams = ["미션과 비전", "창업 스토리", "오른팔", "창업자", "조력자", "브랜드 스토리", "탄생", "경영 철학", "논란", "원칙", "명언"] # 주제 설정
# # 추가 검색어 포함
# plus = {
#     '이케아' : ['앙바르 캄프라르'],
#      '현대' : ['정몽주'],
#       '교보' : ['신용호'],
#       '피자헛' : [],
#       '짜치다' : [],
#       '어메니티' : []}



In [6]:
# 나의 config

# 시스템 설정
debug = False # 디버그모드 설정 :
maxDepth = 2 # depth 설정
limit = 2 # 크롤링 키워드 갯수한계 설정

# 내용 설정
targets = ["넷플릭스", "포르쉐", "넥슨"] # 글감 작성
streams = ["미션과 비전", "창업 스토리", "오른팔", "창업자", "조력자", "브랜드 스토리", "탄생", "경영 철학", "논란", "원칙", "명언"] # 주제 설정
# 추가 검색어 포함
plus = {
    '넷플릭스' : ['앙바르 캄프라르'],
     '현대' : ['정몽주'],
      '교보' : ['신용호'],
      '피자헛' : [],
      '짜치다' : [],
      '어메니티' : []}



In [7]:
print('작업을 시작합니다. 윈도우를 실행합니다.' )
keywords = {} # 데이터베이스 생성
driver = webdriver.Chrome()
tik = time.time()

try :
    for target in targets :    
        # alreadyServeyed = []
        if debug : print('Now Target is : ',target)
        keywords[target] = {}
        subjects = [target+' '+subject for subject in streams]  
        # subjects = [target+' '+subject for subject in streams]  + plus[target]
        searchKeywords = ''        
        for depth in range(maxDepth) :
            if debug : print('now Depth is : ', depth)
            if depth == 0 :
                if debug : print('this is first trial')                
                searchKeywords = subjects.copy()
                print(f'[TARGET : {target} _ DEPTH: {depth}]*******all keywords we search is ', searchKeywords)
            elif depth > 0 :
                befSearched = searchKeywords.copy()
                searchKeywords = list(keywords[target].keys())
                for word in searchKeywords :
                    if word in befSearched : searchKeywords.remove(word)
                print(f'[TARGET : {target} _ DEPTH: {depth}]*******all keywords we search is ', searchKeywords)
            
            if debug : display(pd.DataFrame(keywords[target]))
            # 키워드 분석 시작
            for searchKeyword in searchKeywords : # 주제어에 대한 반복문                
                print('NOW PROCESSING THIS (target, searchKeyword, depth): ',target,searchKeyword,depth)

                # 데이터 수집 - 뷰티풀스프
                daumOpenKeywords = getOpenKeywords(createDaumURL(searchKeyword), daumOpenKeywordsTag, debug)
                daumOpenKeywords = limitLength(daumOpenKeywords, limit, debug)
                if debug : print('Collecting naverOpenKeywords..')
                # naverOpenKeywords = getOpenKeywords(createNaverURL(searchKeyword), naverOpenKeywordsTag)
                if debug : print('daum open keywords complete')

                # 데이터수집 - 셀레니움
                
                try:
                    # 1. 대상 웹사이트 접속
                    driver.get(createDaumURL(searchKeyword))
                    daumSuggestKeywords = seleniumClickAndCrawling(driver, daumClickElementCSSQuery, daumDataCSSQuery, debug)
                    daumSuggestKeywords = limitLength(daumSuggestKeywords, limit,debug)
                    if debug : print('daum suggest keywords complete')
                    driver.get(createGoogleURL(searchKeyword))
                    googleSuggestKeywords = seleniumClickAndCrawling(driver, googleClickElementCSSQuery, googleDataCSSQuery, debug)
                    googleSuggestKeywords = limitLength(googleSuggestKeywords, limit, debug)
                    if debug : print('google suggest keywords complete')
                    # googleSuggestKeywords = [searchKeyword+" "+word for word in googleSuggestKeywords] # 구글 특수처리                    
                    if debug : print('google suggest keywords modifying complete')
                    googleOpenKeywords = getGoogleOpenKeywords(driver,searchKeyword, getGoogleOpenKeywordCSSQuery, debug)
                    googleOpenKeywords = limitLength(googleOpenKeywords, limit, debug)
                    if debug : print('google open keywords complete')
                    
                    
                    
                    # 에러 핸들링
                except Exception as e:
                    if debug : print('[error occured]', e)
                    continue
                # 셀레니움 종료
                # finally:
                #     driver.quit()
                datas = [daumOpenKeywords, daumSuggestKeywords, googleSuggestKeywords, googleOpenKeywords] 
                merged_dict = {}                

                for provider, data in zip(providers, datas) : 
                    if data is not None :     
                        formatWithInfo = keywordValuationFormat.copy()
                        formatWithInfo['depth'] = depth                        
                        formatWithInfo['provider'] = provider                        
                        dictTypeData = keywordFormatting(data, formatWithInfo.copy(), debug)                                          
                        merged_dict.update(dictTypeData)
                        
                
                entire_dict = keywords[target].copy()
                if debug : print('merged_dict', merged_dict)
                if debug : print('entire_dict', entire_dict)
                merged_dict.update(entire_dict)
                if debug : print('merged_dict+', merged_dict)
                keywords[target] = merged_dict.copy()
                if debug : print('keywords', keywords)
                
            
        # 광고갯수 측정 : beautifulsoup                
        print(f"[{target}]키워드 수집이 종료되었습니다. 키워드 분석을 시작합니다.")
        for key, value in keywords[target].items() :
            # if key not in alreadyServeyed :
                count = countDaumADs(createDaumURL(key), key, [DAUM_AD_special,DAUM_AD_normal], debug)
                if debug : print('ad counting complete. number of ad :', count, 'at :', key)
                keywords[target][key]['countDaumADs'] = count
            
        
        #검색게시물 수랑 순서            
        # driver = webdriver.Chrome()
        for key, value in keywords[target].items() :
            try :
                # if key not in alreadyServeyed :
                    time.sleep(1)
                    driver.get(createGoogleURL(key)) # 디렉션
                    volume = countGoogleSearchVolume(driver,volumeCSSQuery, debug)    # 검색량 측정
                    if debug : print('counting search volume complete volume : ', volume, 'at : ',key)
                    rank = checkGoogleSectionOrder(driver, rankCSSQuery, debug) # 랭크 측정                
                    if debug : print('checking tistory rank complete. rank : ', rank, 'at : ',key)
                    keywords[target][key]['searchVolume'] = volume # 검색량 저장
                    keywords[target][key]['tistoryRank'] = rank # 랭크 저장
                    if debug : print('rank and volume saving complete')
            except Exception as e:
                if debug : print('[[error occured]]', e)
        
        pd.DataFrame(keywords[target]).T.to_csv(f'{target}.csv', encoding='cp949')
        print(f'[{target}] 키워드 분석이 종료되었습니다. csv파일로 저장합니다.')
        if debug : display(pd.DataFrame(keywords[target]))
    
finally :
    print('모든 작업이 종료되었습니다. 윈도우를 종료합니다.' )
    driver.quit()
    tok = time.time()
    print(f'작업 소요시간은 {tok-tik} 초입니다.')



작업을 시작합니다. 윈도우를 실행합니다.
[TARGET : 넷플릭스 _ DEPTH: 0]*******all keywords we search is  ['넷플릭스 미션과 비전', '넷플릭스 창업 스토리', '넷플릭스 오른팔', '넷플릭스 창업자', '넷플릭스 조력자', '넷플릭스 브랜드 스토리', '넷플릭스 탄생', '넷플릭스 경영 철학', '넷플릭스 논란', '넷플릭스 원칙', '넷플릭스 명언']
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 미션과 비전 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 창업 스토리 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 오른팔 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 창업자 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 조력자 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 브랜드 스토리 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 탄생 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 경영 철학 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 논란 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 원칙 0
NOW PROCESSING THIS (target, searchKeyword, depth):  넷플릭스 넷플릭스 명언 0
[TARGET :